In [38]:
import torch
from torch import nn
from torch import Tensor
import datetime 
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# number of epoch
num_epochs = 1
# batch size to compute mini-batch
batch_size = 1
# number of pixels in the image 
input_size = 2
# number of possible digit: 0 to 9 
num_class = 1
# small step to find a minima
learning_rate = 0.004
# hidden size
hidden_size = 500

# Fully connected neural network
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_class):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU() 
        self.layer2 = nn.Linear(hidden_size, hidden_size) 
        self.layer3 = nn.Linear(hidden_size, num_class)  
    
    def forward(self, x):
        outputs = self.layer1(x)
        outputs = self.relu(outputs)
        outputs = self.layer2(outputs)
        outputs = self.relu(outputs)
        outputs = self.layer3(outputs)
        return outputs
    

In [39]:
# function to train each model
def train_model(model_, my_train_input_, my_train_target_, criterion_, optimizer_,num_epochs_,batch_size_):
    # getting start time of train to get the train time at the end thanks to "end_time"
    start_time = datetime.datetime.now()
    # list to get train errors at each epoch
    train_error = []
    # train function
    for epoch in range(1, num_epochs_+1):
        # using technique of mini batch (size of the batch in the function's parameters)
        for i in range(int(len(my_train_input_)/batch_size_)):  
            # getting images and labels in right format
            x = my_train_input_.narrow(0,i*batch_size_,batch_size_).to(device)
            labels = my_train_target_.narrow(0,i*batch_size_,batch_size_).to(device)

            # Forward pass
            outputs = model_(x)
            loss = criterion_(outputs, labels)

            # Backward and optimize
            optimizer_.zero_grad()
            loss.backward()
            optimizer_.step()
            # getting train error at each epoch
            train_error.append(test_accuracy(model_, my_train_input_, my_train_target_))
        # getting end time and training time
        end_time = datetime.datetime.now()
        training_time = end_time - start_time
        print ('Loss: {:.4f} on epoch: {}, train error: {:.5f}'.format(loss.item(),epoch,train_error[-1]))
    return train_error, training_time

In [46]:
def test_accuracy(model_, my_test_input_, my_test_target_):
    total = my_test_input_.size(0)
    outputs = model_(my_test_input_)
    print(outputs)
    print(my_test_target_)
    well_predicted_count = (outputs == my_test_target_).sum().item()

    return 1 - well_predicted_count / total

In [47]:
import numpy
data_x = numpy.matrix([[469,3.21],[294,3.27],[202,3.2],[154,3.28],[119,3.4],[97,3.23],[76, 3.3],[66,3],[59,2.95],[51,3]])
data_y = numpy.matrix([[30],[50],[70],[100],[130],[160],[200],[230],[260],[300]])
input_ = torch.from_numpy(data_x)
target_ = torch.from_numpy(data_y)

print(input_)
print(target_)

tensor([[469.0000,   3.2100],
        [294.0000,   3.2700],
        [202.0000,   3.2000],
        [154.0000,   3.2800],
        [119.0000,   3.4000],
        [ 97.0000,   3.2300],
        [ 76.0000,   3.3000],
        [ 66.0000,   3.0000],
        [ 59.0000,   2.9500],
        [ 51.0000,   3.0000]], dtype=torch.float64)
tensor([[ 30],
        [ 50],
        [ 70],
        [100],
        [130],
        [160],
        [200],
        [230],
        [260],
        [300]], dtype=torch.int32)


In [48]:
model = NeuralNet(input_size, hidden_size, num_class).to(device)
criterion = nn.L1Loss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)  
train_error, training_time = train_model(model, input_.float(), target_.float(), criterion, optimizer,num_epochs,batch_size)

tensor([[4000.0120],
        [2508.3938],
        [1724.1837],
        [1315.1006],
        [1016.8470],
        [ 829.2392],
        [ 650.3024],
        [ 564.8984],
        [ 505.2095],
        [ 437.0632]], grad_fn=<AddmmBackward>)
tensor([[ 30.],
        [ 50.],
        [ 70.],
        [100.],
        [130.],
        [160.],
        [200.],
        [230.],
        [260.],
        [300.]])
tensor([[-126.5109],
        [ -79.3566],
        [ -54.5590],
        [ -41.6264],
        [ -32.2001],
        [ -26.2603],
        [ -20.6022],
        [ -17.8876],
        [ -15.9972],
        [ -13.8398]], grad_fn=<AddmmBackward>)
tensor([[ 30.],
        [ 50.],
        [ 70.],
        [100.],
        [130.],
        [160.],
        [200.],
        [230.],
        [260.],
        [300.]])
tensor([[3.0825],
        [1.9694],
        [1.3827],
        [1.0792],
        [0.8591],
        [0.7159],
        [0.5874],
        [0.5187],
        [0.4770],
        [0.4336]], grad_fn=<AddmmBackward>)


In [35]:
model(Tensor([51,3]))

tensor([14.7696, 12.9990, 13.5972, 14.1123, 17.0108, 22.6005, 13.5195, 16.7820,
        13.8601, 17.7458, 15.8322, 13.6685, 13.9426, 20.3678, 15.1451, 12.9439,
        14.3262, 20.6364, 15.6986, 14.9710, 15.8282, 13.2200, 15.4951, 18.4752,
        15.1390, 15.0656, 14.9025, 14.2051, 15.9589, 18.3631, 13.9445, 15.4239,
        13.7188, 15.5536, 14.3146, 15.3523, 16.3967, 22.0201, 15.6569, 12.4195,
        14.2364, 14.1782, 14.7375, 12.3826, 13.4828, 12.4341, 17.5544, 14.2052,
        14.6860, 13.8939, 20.0757, 15.0123, 15.1471, 15.8959, 13.5900, 16.2137,
        13.0337, 14.8032, 13.8524, 14.4916, 14.4134, 14.8064, 12.4903, 13.8422,
        14.3710, 15.3202, 13.7717, 13.1972, 12.6730, 14.2258, 15.0868, 14.8226,
        21.6567, 14.0519, 15.8046, 18.9750, 13.0635, 14.6599, 18.0238, 17.3375,
        14.8592, 14.7172, 13.3466, 18.3617, 14.0268, 13.5351, 13.3650, 14.2281,
        14.1835, 15.4894, 14.1583, 15.0794, 12.6972, 13.7437, 15.8107, 13.5619,
        26.0594, 16.4941, 17.7803, 13.02